
# Determining and plotting the altitude/azimuth of a celestial object including magnitudes and types of objects for September 2022

This example demonstrates coordinate transformations and the creation of
visibility curves to assist with observing run planning.

In this example, we make a `~astropy.coordinates.SkyCoord` instance for M33.
The altitude-azimuth coordinates are then found using
`astropy.coordinates.EarthLocation` and `astropy.time.Time` objects.

This example is meant to demonstrate the capabilities of the
`astropy.coordinates` package. For more convenient and/or complex observation
planning, consider the `astroplan <https://astroplan.readthedocs.org/>`_
package.


*By: Erik Tollerud, Kelle Cruz*

*License: BSD*


- Author Sylvie Dagoret-Campagne
- Creation : September 12th 2022
- Update 

Let's suppose you are planning to visit picturesque Bear Mountain State Park
in New York, USA. You're bringing your telescope with you (of course), and
someone told you M33 is a great target to observe there. You happen to know
you're free at 11:00 pm local time, and you want to know if it will be up.
Astropy can answer that.

Import numpy and matplotlib. For the latter, use a nicer set of plot
parameters and set up support for plotting/converting quantities.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, quantity_support
plt.style.use(astropy_mpl_style)
quantity_support()

%matplotlib inline

Import the packages necessary for finding coordinates and making
coordinate transformations



In [82]:
import astropy.units as u
from astropy.coordinates import Angle
from astropy.time import Time, TimezoneInfo
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

In [3]:
from astroquery.simbad import Simbad

In [4]:
from astropy.table import Table

In [5]:
import matplotlib.colors as colors
import matplotlib.cm as cmx

In [44]:
from datetime import datetime
import timezonefinder, pytz
from calendar import monthrange
import calendar

- label lines
https://github.com/cphyc/matplotlib-label-lines

In [8]:
#from labellines import labelLine, labelLines

In [9]:
plt.rcParams["axes.labelsize"]="large"
plt.rcParams["axes.linewidth"]=2.0
plt.rcParams["xtick.major.size"]=8
plt.rcParams["ytick.major.size"]=8
plt.rcParams["ytick.minor.size"]=5
plt.rcParams["xtick.labelsize"]="large"
plt.rcParams["ytick.labelsize"]="large"

plt.rcParams["figure.figsize"]=(12,8)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
#plt.rcParams['axes.facecolor'] = 'blue'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['lines.markeredgewidth'] = 0.3 # the line width around the marker symbol
plt.rcParams['lines.markersize'] = 5  # markersize, in points
plt.rcParams['grid.alpha'] = 0.75 # transparency, between 0.0 and 1.0
plt.rcParams['grid.linestyle'] = '-' # simple line
plt.rcParams['grid.linewidth'] = 0.4 # in points
plt.rcParams['font.size'] = 13

# Initialisation:

## Observation location

In [10]:
LSSTNAME='Cerro Pachon'
lsst_location=EarthLocation.of_site(LSSTNAME)
longitude=lsst_location.lon
latitude=lsst_location.lat
altitude=lsst_location.height

## Determination of UTC offset from Time zone in Chile

In [11]:
tf = timezonefinder.TimezoneFinder()

In [12]:
timezone_str = tf.certain_timezone_at(lat=latitude.degree, lng=longitude.degree)

In [13]:
timezone_str 

'America/Santiago'

In [45]:
if timezone_str is None:
    print("Could not determine the time zone")
else:
    # Display the current time in that time zone
    
    #tz = pytz.timezone('Europe/Berlin')
    #berlin_now = datetime.now(tz)
    
    timezone = pytz.timezone(timezone_str)
    timezone0 = pytz.timezone('UTC')
    
    dt = datetime.utcnow()
    timeChile_now = datetime.now(timezone)
    
    print("The UTC Time now %s" % dt)
    print("The actual time in %s is %s" % (timezone_str, dt + timezone.utcoffset(dt)))

The UTC Time now 2022-09-10 18:24:21.491912
The actual time in America/Santiago is 2022-09-10 15:24:21.491912


In [46]:
utcoffset=timezone.utcoffset(dt).total_seconds()/60./60.
print("The UTC offset in Chile is ",utcoffset," hours")

The UTC offset in Chile is  -3.0  hours


In [61]:
tz_utc_minus_three_hours = TimezoneInfo(utc_offset=int(utcoffset)*u.hour)
tz_utc_minus_three_hours

## Initialisation of Simbad

In [16]:
# to view the list of VOTABLE
# Simbad.list_votable_fields()

In [48]:
Simbad.add_votable_fields('sptype')
Simbad.add_votable_fields('flux(U)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(B)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(V)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(R)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(I)') # add the Vega unit Magnitude 
#Simbad.add_votable_fields('flux_unit(V)')
#Simbad.add_votable_fields('flux_unit(I)')
#Simbad.add_votable_fields('flux_system(V)')
#Simbad.add_votable_fields('flux_system(I)')
#Simbad.add_votable_fields('ubv')  # Johnson UBV system

## Targets

In [49]:
all_target_names_old= ["hd111980","hd101452","hd115169","hd142331",'FEIGE110', 'GD108', 'GD50',
                   'HD009051', 'HD031128','HD074000', 'HD111980', 'HD14943', 'HD160617', 'HD185975',
                   'HD200654', 'HD205905', 'HD37962', 'HD38949', 'HD60753','LTT9491', 'NGC7293', 'SIRIUS', 
                   'WD0308-565','HD009051','HD14943','HD002811','PNG321.0+03.9','HD2811','HD009051','HD14943',
                   'HD031128','LAM LEP','HD37962','mu. Col','HD38949','ETA1 DOR','HD60753','HD074000','HD160617',
                   'HD167060','HD185975','HD200654','HD205905','HD14943','HD031128','LAM LEP','HD37962','mu. Col',
                   'HD38949','ETA1 DOR','HD60753','HD074000','HD111980','HD115169','BD-11 192','HD142331','18SCO',
                   'HD160617','HD167060','HD185975']

all_target_names = ['18SCO', 'BD-11 192', 'ETA1 DOR', 'FEIGE110', 'GD108', 'GD50','HD002811', 'HD009051',
                    'HD031128', 'HD074000', 'HD111980','HD115169', 'HD142331', 'HD14943', 'HD160617','HD167060',
                    'HD185975', 'HD200654', 'HD205905', 'HD2811', 'HD37962', 'HD38949','HD60753', 'LAM LEP',
                    'LTT9491', 'NGC7293', 'PNG321.0+03.9','SIRIUS', 'WD0308-565', 'hd101452', 'hd111980',
                    'hd115169','hd142331', 'mu. Col']

NTargets=len(all_target_names)

In [50]:
print(f"Number of targets : {NTargets}")

Number of targets : 34


## Get coordinates

In [51]:
all_target_locations = [ SkyCoord.from_name(target_name) for target_name in all_target_names]

In [52]:
all_ra=[]
all_dec=[]
for target_loc in all_target_locations:
    all_ra.append(target_loc.ra)
    all_dec.append(target_loc.dec)

## Find magnitudes

In [53]:
all_Vmag = []
all_types = []
all_target_tagnames = []
for target_name in all_target_names:
    result_table = Simbad.query_object(target_name)
    all_Vmag.append(result_table['FLUX_V'][0])
    all_types.append(result_table['SP_QUAL'][0]) 
    tagname =  target_name + ", t=" + result_table['SP_QUAL'][0] + ", m=" + str(result_table['FLUX_V'][0])
    print(tagname)
    all_target_tagnames.append(tagname)

18SCO, t=B, m=5.5
BD-11 192, t=C, m=9.993
ETA1 DOR, t=C, m=5.69
FEIGE110, t=B, m=11.5
GD108, t=D, m=13.563
GD50, t=C, m=14.063
HD002811, t=C, m=7.5
HD009051, t=D, m=8.92
HD031128, t=D, m=9.14
HD074000, t=D, m=9.66
HD111980, t=C, m=8.38
HD115169, t=C, m=9.2
HD142331, t=C, m=8.75
HD14943, t=C, m=5.898
HD160617, t=D, m=8.73
HD167060, t=C, m=8.92
HD185975, t=C, m=8.1
HD200654, t=C, m=9.11
HD205905, t=C, m=6.74
HD2811, t=C, m=7.5
HD37962, t=C, m=7.85
HD38949, t=C, m=7.8
HD60753, t=C, m=6.68
LAM LEP, t=C, m=4.29
LTT9491, t=C, m=14.111
NGC7293, t=C, m=13.524
PNG321.0+03.9, t=C, m=11.64
SIRIUS, t=C, m=-1.46
WD0308-565, t=D, m=14.14
hd101452, t=E, m=8.2
hd111980, t=C, m=8.38
hd115169, t=C, m=9.2
hd142331, t=C, m=8.75
mu. Col, t=C, m=5.18


## Build table

In [78]:
from astropy.table import Table
t = Table()
t["target"] = all_target_names
t["types"] = all_types
t["Vmag"]  = all_Vmag 
t["ra"]    = all_ra 
t["dec"]    = all_dec 

In [79]:
t['ra'].info.format = '9.6f'
t['dec'].info.format = '9.6f'
t['Vmag'].info.format = '4.2f'

In [80]:
t

target,types,Vmag,ra,dec
,,,deg,deg
str13,str1,float32,float64,float64
18SCO,B,5.50,243.905293,-8.369439
BD-11 192,C,9.99,15.588059,-10.423859
ETA1 DOR,C,5.69,91.539090,-66.039620
FEIGE110,B,11.50,349.993331,-5.165603
GD108,D,13.56,150.196859,-7.558549
GD50,C,14.06,57.209109,-0.975636
HD002811,C,7.50,7.827041,-43.606389
...,...,...,...,...


In [54]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NTargets)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NTargets), alpha=1)

## Observation Date

In [55]:
NYEAR=2022
NMONTH=9
NDAY=12

In [64]:
night_obs = datetime(NYEAR, NMONTH, NDAY, 0, 0)
night_obs_midnight= datetime(NYEAR, NMONTH, NDAY, 23, 59,59,tzinfo = tz_utc_minus_three_hours)

In [57]:
night_obs_midnight

datetime.datetime(2022, 9, 12, 23, 59, 59)

In [58]:
night_obs_str=night_obs.strftime("%Y-%m-%d %H:%M:%S")
night_obs_str

'2022-09-12 00:00:00'

In [59]:
night_obs_midnight_str=night_obs_midnight.strftime("%Y-%m-%d %H:%M:%S")
night_obs_midnight_str

'2022-09-12 23:59:59'

## Compute UTC and Sideral Time

In [66]:
tz_utc = TimezoneInfo() 
print("Local time in Standiago in summer (GMT-3) : ",night_obs_midnight)
print("Time in UTC                               : ",night_obs_midnight.astimezone(tz_utc))

Local time in Standiago in summer (GMT-3) :  2022-09-12 23:59:59-03:00
Time in UTC                               :  2022-09-13 02:59:59+00:00


In [71]:
t_lsst = Time(night_obs_midnight.astimezone(tz_utc), scale='utc',location=lsst_location)
t_sidereal_lsst = t_lsst.sidereal_time('apparent')  
t_sidereal_lsst

<Longitude 21.75662374 hourangle>

In [85]:
Angle(t_sidereal_lsst).to(u.degree)

<Angle 326.34935608 deg>

In [86]:
Angle(t_sidereal_lsst).degree

326.34935607926764

In [74]:
t_lsst = Time('2022-09-13 02:59:59', scale='utc',location=('-70.73668333d', '-30.24074167d'))
t_lsst.sidereal_time('apparent') 

<Longitude 21.75662374 hourangle>

In [84]:
Angle(t_sidereal_lsst).to(u.degree)

<Angle 326.34935608 deg>

## Filter the rows

In [97]:
MarginAngle = Angle(10.0,u.hour)
MarginAngle

<Angle 10. hourangle>

In [106]:
all_flag_select = []
for row in t:
    ra_angle = Angle(row['ra'],u.degree)
    ra_min = t_sidereal_lsst - MarginAngle
    ra_max = t_sidereal_lsst + MarginAngle
    print(row['ra'],ra_angle.hour)
    print(ra_min,ra_max)
    flag = ra_angle.is_within_bounds([ra_min,ra_max])
    all_flag_select.append(flag)                

243.90529281 16.260352854000004
11h45m23.84545902s 31h45m23.84545902s
15.58805914 1.0392039426666668
11h45m23.84545902s 31h45m23.84545902s
91.53908976 6.102605984000001
11h45m23.84545902s 31h45m23.84545902s
349.99333127 23.33288875133334
11h45m23.84545902s 31h45m23.84545902s
150.19685918 10.013123945333335
11h45m23.84545902s 31h45m23.84545902s
57.20910905 3.813940603333334
11h45m23.84545902s 31h45m23.84545902s
7.82704127 0.5218027513333334
11h45m23.84545902s 31h45m23.84545902s
22.19376105 1.4795840700000003
11h45m23.84545902s 31h45m23.84545902s
73.04129062 4.869419374666667
11h45m23.84545902s 31h45m23.84545902s
130.21168267 8.680778844666667
11h45m23.84545902s 31h45m23.84545902s
193.31272008 12.887514672000002
11h45m23.84545902s 31h45m23.84545902s
198.9474513 13.263163420000003
11h45m23.84545902s 31h45m23.84545902s
238.58245199 15.905496799333337
11h45m23.84545902s 31h45m23.84545902s
35.72781358 2.3818542386666675
11h45m23.84545902s 31h45m23.84545902s
265.705515 17.713701000000004
11h4

In [107]:
t["select"] = all_flag_select

In [108]:
t["select"] == True

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])

## Time zone at observation

In [ ]:
if timezone_str is None:
    print("Could not determine the time zone")
else:
    # Display the current time in that time zone
    
    #tz = pytz.timezone('Europe/Berlin')
    #berlin_now = datetime.now(tz)
    
    timezone = pytz.timezone(timezone_str)
    timezone0 = pytz.timezone('UTC')
    
    dt = night_obs_midnight
    
    print("The UTC Time at observation %s" % dt)
    print("The actual time in %s is %s" % (timezone_str, dt + timezone.utcoffset(dt)))

In [ ]:
utcoffset=timezone.utcoffset(dt).total_seconds()/60./60.
print("The UTC offset in Chile is ",utcoffset," hours")

In [ ]:
utcoffset=utcoffset*u.hour  # need hours units

## Observation Location

In [ ]:
LSSTNAME='Cerro Pachon'
lsst_location=EarthLocation.of_site(LSSTNAME)
longitude=lsst_location.lon
latitude=lsst_location.lat
altitude=lsst_location.height

In [ ]:
cerro_pachon = EarthLocation(lat=latitude.degree*u.deg, lon=longitude.degree*u.deg, height=altitude)

## Check Position of target at midnight

http://catserver.ing.iac.es/staralt/

# Observation frame

In [ ]:
midnight_utc = Time(night_obs_midnight_str) - utcoffset
delta_midnight = np.linspace(-2, 10, 100)*u.hour
frame_night = AltAz(obstime=midnight_utc+delta_midnight,location=cerro_pachon)

In [ ]:
delta_midnight = np.linspace(-12, 12, 1000)*u.hour
times_evening_to_morning = midnight_utc + delta_midnight
frame_evening_to_morning = AltAz(obstime=times_evening_to_morning, location=cerro_pachon)

### Sun frame

In [ ]:
from astropy.coordinates import get_sun
sunaltazs_evening_to_morning = get_sun(times_evening_to_morning).transform_to(frame_evening_to_morning)

### Moon frame

In [ ]:
from astropy.coordinates import get_moon
moon_evening_to_morning = get_moon(times_evening_to_morning)
moonaltazs_evening_to_morning = moon_evening_to_morning.transform_to(frame_evening_to_morning)

### Target frame

In [ ]:
all_target_altazs_evening_to_morning = [target_location.transform_to(frame_evening_to_morning) for target_location in  all_target_locations ]

# Order according increasing maxinum rise time

In [ ]:
from astropy.coordinates import Angle

In [ ]:
all_timemax=np.zeros(NTargets)
all_altitudesmax=np.zeros(NTargets)
for idx in np.arange(NTargets):
    altitudes=all_target_altazs_evening_to_morning[idx].alt
    idx_max=np.where(altitudes==altitudes.max())[0][0]
    all_timemax[idx]=delta_midnight[idx_max].value
    all_altitudesmax[idx]=altitudes.max().degree

In [ ]:
Angle(altitudes.max()).degree

In [ ]:
all_timemax

In [ ]:
sorted_indexes=np.argsort(all_timemax)

In [ ]:
sorted_all_timemax=sorted(all_timemax)
sorted_all_timemax

# Plot

In [ ]:
plt.figure(figsize=(20,12))
plt.plot(delta_midnight, sunaltazs_evening_to_morning.alt, color='r', ls=":",label='Sun',lw=5)
plt.plot(delta_midnight, moonaltazs_evening_to_morning.alt, color=[0.75]*3, ls='--', label='Moon',lw=5)

rank=0
for idx in sorted_indexes:
    label=str(rank)+" : "+all_target_tagnames[idx]
    plt.plot(delta_midnight, all_target_altazs_evening_to_morning[idx].alt,label=label, lw=2.5,color=all_colors[rank])

    if all_target_altazs_evening_to_morning[idx].alt[-1]>0:
        plt.text(delta_midnight[-1], all_target_altazs_evening_to_morning[idx].alt[-1], f'{rank}',color=all_colors[rank],fontsize=20)
    if all_target_altazs_evening_to_morning[idx].alt[0]>0:
        plt.text(delta_midnight[0], all_target_altazs_evening_to_morning[idx].alt[0], f'{rank}',color=all_colors[rank],fontsize=20)
    if all_altitudesmax[idx]:
        plt.text(all_timemax[idx], all_altitudesmax[idx], f'{rank}',color=all_colors[rank],fontsize=20)
    
    rank+=1
    
#labelLines(plt.gca().get_lines(), zorder=2.5)
#labelLines(plt.gca().get_lines(), align=False, fontsize=14)
#xvals=sorted(all_timemax)
#labelLines(plt.gca().get_lines(), align=False, xvals=xvals, color="k")
#labelLines(plt.gca().get_lines(), xvals=(-10., 10.), zorder=2.5)


plt.fill_between(delta_midnight, 0*u.deg, 90*u.deg,sunaltazs_evening_to_morning.alt < -0*u.deg, color='0.5', zorder=0)
plt.fill_between(delta_midnight, 0*u.deg, 90*u.deg,sunaltazs_evening_to_morning.alt < -18*u.deg, color='k', zorder=0)

plt.legend(loc='upper left')
plt.xlim(-12*u.hour, 12*u.hour)
plt.xticks((np.arange(13)*2-12)*u.hour)
plt.ylim(0*u.deg, 90*u.deg)
plt.xlabel('Hours from Midnight local time')
plt.ylabel('Altitude [deg]')
title = "Observations at AuxTel at night " + night_obs_str.split(" ")[0]
plt.title(title)

figname="AuxtelStarAlt_{:4d}_{:d}_{:d}.png".format(NYEAR,NMONTH,NDAY)

plt.savefig(figname)
plt.show()

# Create astropy Table

In [ ]:
t = Table()

In [ ]:
t["name"] = all_target_names
t["ra"] = all_ra
t["dec"] = all_dec
t["type"] = all_types 
t["Vmag"] = all_Vmag  
t["tculmin"] = all_timemax

In [ ]:
t["ra"].info.format ='5.3f'
t["dec"].info.format ='5.3f'
t["Vmag"].info.format ='5.3f'
t["tculmin"].info.format ='5.3f'

In [ ]:
t.show_in_notebook()

In [ ]:
t